In [64]:
import pandas as pd
import numpy as np
from pathlib import Path
from fastai.text import *

In [65]:
DATA_PATH = Path('data')
DATA_PATH

WindowsPath('data')

## Data

In [66]:
dataset = pd.read_json(DATA_PATH/'train.json')
dataset.head()

,comments,description,id,sql,sql_plain,title,url
0,[],,586912,"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",Repent 151-160,http://data.stackexchange.com/stackoverflow/re...
1,[This is test data for my question to make thi...,,54791,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...
2,[],,611565,select Id from Comments where lower(Text) like...,select Id\nfrom Comments \nwhere lower(Text) l...,Accept answer,http://data.stackexchange.com/stackoverflow/re...
3,[],,612812,"DECLARE @From DATETIME = convert(DATETIME, '##...","DECLARE @From DATETIME = convert(DATETIME, '##...",Questions with most views created within 3 mon...,http://data.stackexchange.com/stackoverflow/re...
4,"[Most Prolific Answerers (with score), Shows t...",,325114,"SELECT TOP 50 OwnerUserId as [User Link], COUN...",-- Most Prolific Answerers (with score)\n-- Sh...,Most Prolific Answers,http://data.stackexchange.com/stackoverflow/re...


In [146]:
SQL = dataset['sql']

In [147]:
## Transform the text to be tokenize readily
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? '}
    

In [148]:
def fixup(x):
    for bereplaced, replace in token_replaced.items():        
        x = x.replace(bereplaced, replace)
    return x

In [149]:
## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? ',
                 ')' : ' ) ',
                 '(' : ' ( ',
                 '=' : ' = ',
                 '/' : ' / ',
                 '*' : ' * '}
    


In [150]:
SQL = SQL.apply(fixup)

In [154]:
## Sanity Check
SQL.iloc[0]

In [156]:
SQL.iloc[1]

"CREATE TABLE  # CustomerDB  (  ID int NOT NULL ,  NAME nvarchar ( 50 )  NULL ,  COMPANY nvarchar ( 50 )  NULL ,  ADDR1 nvarchar ( 50 )  NULL ,   )  INSERT INTO  # CustomerDB ( ID ,  NAME ,  COMPANY ,  ADDR1 )  VALUES  ( 16242 , 'TOM E' , 'Company A' , '101 First RD' )  ,   ( 16241 , 'RONALD J' , 'Company B' , '12 Tenth AVE' )  ,   ( 16235 , 'KENNETH H' , '' , '12 Tenth AVE' )  ,   ( 16238 , 'MICHAEL H' , 'Company C' , '12 Tenth AVE' )  ,   ( 16243 , 'ANTONIO D' , 'Company D' , '264 Long ST STE 5' )  ,   ( 16237 , 'MICHAEL B' , 'Company E' , '264 Long ST STE 5' )  ,   ( 16234 , 'WALTER L' , 'Company F' , '73 North RD' )  ,   ( 16236 , 'CARL O' , 'Company G' , '73 North RD' )  ,   ( 16239 , 'MICHAEL S' , 'Company H' , '73 North RD' )  ,   ( 16240 , 'MICHAEL I' , 'Company I' , '73 North RD' )  SELECT c1 . id ,  c1 . name ,  c1 . company ,  c1 . addr1 FROM  # CustomerDB c1 LEFT OUTER JOIN  # CustomerDB c2 ON  ( c1 . addr1 =  c2 . addr1 )  GROUP BY c1 . addr1 ,  c1 . id ,  c1 . name ,  c1 

In [155]:
SQL.iloc[100]

"select a . month ,  total ,  unanswered ,  cast ( cast ( unanswered*1 . 0/total*100 as decimal ( 5 , 1 )  )  as varchar ( 5 )  )  + ' % ' as PercentUnanswered from  ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count ( * )  as total from posts where posttypeid  =  1 group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  a ,   ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count ( * )  as unanswered from posts where postTypeID  =  1 and id not in  (  select distinct ParentId from posts where PostTypeId  =  2 )  group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  b where a . month  =  b . month order by a . month"

In [160]:
sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 
# sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL.iloc[:10])) # For Testing

In [158]:
tmp[0]

[' ',
 't_up',
 'select',
 'p',
 '.',
 'title',
 ',',
 'p',
 '.',
 'score',
 ',',
 'p',
 '.',
 'viewcount',
 ',',
 'p',
 '.',
 'answercount',
 ',',
 'p',
 '.',
 'commentcount',
 ',',
 't_up',
 'len',
 '(',
 'p',
 '.',
 'body',
 ')',
 'as',
 'ques_size',
 ',',
 'p',
 '.',
 'favoritecount',
 ',',
 'p',
 '.',
 'i',
 'd',
 ',',
 'u',
 '.',
 'reputation',
 'as',
 'user_repo',
 ',',
 'u',
 '.',
 'creationdate',
 'as',
 'user_join_date',
 ',',
 'datediff',
 '(',
 't_up',
 'minute',
 ',',
 'p',
 '.',
 'creationdate',
 ',',
 'a',
 '.',
 'creationdate',
 ')',
 'as',
 'qatimegap',
 'from',
 'posts',
 'as',
 'p',
 ',',
 'posts',
 'as',
 'a',
 ',',
 'users',
 'as',
 'u',
 'where',
 'p',
 '.',
 'i',
 'd',
 '=',
 'a',
 '.',
 'parentid',
 'and',
 'p',
 '.',
 'owneruserid',
 '=',
 'u',
 '.',
 'i',
 'd',
 'and',
 'a',
 '.',
 'creationdate',
 '=',
 '(',
 'select',
 'min',
 '(',
 'tau',
 '.',
 'creationdate',
 ')',
 'from',
 'posts',
 'tau',
 'where',
 'tau',
 '.',
 'parentid',
 '=',
 'p',
 '.',
 'i',
 'd

## Tokenize

In [161]:
np.percentile([len(o) for o in sql_tok], 90)

199.0

In [163]:
keep = np.array([len(o)<30 for o in sql_tok])

In [164]:
sql_tok = np.array(sql_tok)[keep]

In [167]:
# pickle.dump(sql_tok, (DATA_PATH/'SQL_tok.pkl').open('wb'))

In [170]:
sql_tok = pickle.load((DATA_PATH/'sql_tok.pkl').open('rb'))

In [173]:
sql_tok.shape

(5081,)

## Model

## Inference

##  State Machine

## Generate text base on Inference and state machine